In [43]:
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
import pygwalker as pyg

In [92]:
file_paths_sugar_bag = {
    'weather_station': './SugarBag/SugarBagRd_Atmos_Aggregated.csv',
    'wifi_counter': './SugarBag/SugarBagRd_NCount_Aggregated.csv',
    'object_detection': './SugarBag/Alpha_X_Aggregated.csv',
    'infrared_bi_directional': './SugarBag/SugarBagRd_Farmo_Aggregated.csv',
    'infrared_counters': './SugarBag/SugarBagRd_DingTek_Aggregated.csv',
    'vibration_counters': './SugarBag/SugarBagRd_R718mbb_Aggregated.csv'

}

def load_dataset(file_path):
    return pd.read_csv(file_path)

sugar_bag_data = { sensor: load_dataset(path) for sensor, path in file_paths_sugar_bag.items()}

local_time = pd.to_datetime("2023-01-21 10:00:00")

In [94]:
trail_info = pd.DataFrame({"Name":["Aero","Bees Knees","Black Schute","Boner log line","Bottom Powerline Access Trail","Fantales","Golden Rough","Honeycomb","Milky Way","Party mix","Playground","Playground climb trail","Rocky Road","Sour Power","Sweet Sugar","Syrup","The Drop In Clinic","Willy Wonka"],
                               "Difficulty":["Double Black","Blue","Black","Black","Green","Green","Black","Blue","Green","Green","Double Black","Blue","Black","Green","Blue","Blue","Double Black","Blue"],
                               "TrailType":["Freeride","Freeride","Shortcut","Shortcut","Access Trail","Freeride","Freeride","Freeride","Access Trail","Freeride","Freeride/Multiline","Access Trail","Freeride","Access Trail","Freeride","Freeride","Freeride","Freeride"],
                               "Device":["NA","dc500-385c","dc500-4402","NA","NA","NA","fpc-4f48","NA","fpc-4f48","dc500-540f","fpc-da3a?","NA","NA","fpc-da3a","NA","fpc-7559","NA","NA"],
                               "Counter":["NA","nonNegPeopleCount","nonNegPeopleCount","NA","NA","NA","nonNegDifCount2","NA","nonNegDifCount1","nonNegPeopleCount","nonNegDifCount2?","NA","NA","nonNegDifCount1","NA","nonNegDifCount1","NA","NA"]})


OPEN AND REMOVE UNNECESSARY DATA

In [45]:
""" please update these as reqired
"""

def open_and_clean_Weather(): # Updated connor's version
    df =sugar_bag_data['weather_station']
    # Format time
    df['datetime'] = pd.to_datetime(df['datetime'])

    if df.isnull().values.any():
        df.ffill( inplace = True)

    # Drop unneeded columns
    if 'airTemeratureDiff' in df:
        df.drop(columns = ['airTemeratureDiff'], inplace = True)
    if 'atmosphericPressureDiff' in df:
        df.drop(columns = ['atmosphericPressureDiff'], inplace = True)
    if 'gustSpeedDiff' in df:
        df.drop(columns = ['gustSpeedDiff'], inplace = True)
    if 'relativeHumidityDiff' in df:
        df.drop(columns = ['relativeHumidityDiff'], inplace = True)
    if 'windSpeedDiff' in df:
        df.drop(columns = ['windSpeedDiff'], inplace = True)
    if 'windDirectionDiff' in df:
        df.drop(columns = ['windDirectionDiff'], inplace = True)
    if 'windDirection' in df:
        df.drop(columns = ['windDirection'], inplace = True)
    
    #Remove duplicate entries
    df = df.drop_duplicates()


    return df

def open_and_clean_WIFI(): #connor's version
    # open data
    df =sugar_bag_data['wifi_counter']
    # Formatting time
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Check if data frame has null values
    if df.isnull().values.any():
        df.ffill( inplace = True)

    # Drop uneeded columns
    if 'currentDiff' in df:
        df.drop(columns = ['currentDiff'], inplace = True)
    if 'newDiff' in df:
        df.drop(columns = ['newDiff'], inplace = True)
    if 'totalDiff' in df:
        df.drop(columns = ['totalDiff'], inplace = True)

    # Remove duplicate entries
    df = df.drop_duplicates()

    # Remove invalid entries
    for index, row in df.iterrows():
        if row['new'] < 0:
            df.loc[index, 'new'] = 0
        if row['current'] < 0:
            df.loc[index, 'current'] = 0
        if row['total'] < 0:
            df.loc[index, 'total'] = 0

    return df

def open_and_clean_OBJ(): # Updated connor's version
    # open data
    df =sugar_bag_data['object_detection']
    # Formatting time
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Check if data frame has null values
    if df.isnull().values.any():
        df.ffill( inplace = True)

    # Drop uneeded columns
    if 'F1 - peopleLeft/Up' in df:
        df.drop(columns = ['F1 - peopleLeft/Up'], inplace = True)
    if 'F2 - peopleRight/Down' in df:
        df.drop(columns = ['F2 - peopleRight/Down'], inplace = True)
    if 'F9 - trucksLeft/Up' in df:
        df.drop(columns = ['F9 - trucksLeft/Up'], inplace = True)
    if 'F10 - trucksRight/Down' in df:
        df.drop(columns = ['F10 - trucksRight/Down'], inplace = True)
  

    # Remove duplicate entries
    df = df.drop_duplicates()

    # Remove invalid entries
    for index, row in df.iterrows():
        if row["F3 - bikesLeft/Up"] < 0:
            df.loc[index, 'F3 - bikesLeft/Up'] = 0
        if row['F4 - bikesRight/Down'] < 0:
            df.loc[index, 'F4 - bikesRight/Down'] = 0
        if row["F5 - carsLeft/Up"] < 0:
            df.loc[index, 'F5 - carsLeft/Up'] = 0
        if row['F6 - carsRight/Down'] < 0:
            df.loc[index, 'F6 - carsRight/Down'] = 0
        if row["F7 - busesLeft/Up"] < 0:
            df.loc[index, 'F7 - busesLeft/Up'] = 0
        if row['F8 - busesRight/Down'] < 0:
            df.loc[index, 'F8 - busesRight/Down'] = 0
        


    return df

def open_and_clean_IRBD():
    df =sugar_bag_data["infrared_bi_directional"]
    # Formatting time
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Check if data frame has null values
    if df.isnull().values.any():
        df.ffill( inplace = True)
    
    # Drop uneeded columns
    if 'battery' in df:
        df.drop(columns = ['battery'], inplace = True)
    if 'count1' in df:
        df.drop(columns = ['count1'], inplace = True)
    if 'count2' in df:
        df.drop(columns = ['count2'], inplace = True)
    if 'activity' in df:
        df.drop(columns = ['activity'], inplace = True)

    # Remove duplicate entries
    df = df.drop_duplicates()

    # Remove invalid entries and cap entries at 200
    for index, row in df.iterrows():
        if row['nonNegDifCount1'] < 0:
            df.loc[index, 'nonNegDifCount1'] = 0
        elif row['nonNegDifCount1'] > 200:
            df.loc[index, 'nonNegDifCount1'] = 200
        if row['nonNegDifCount2'] < 0:
            df.loc[index, 'nonNegDifCount2'] = 0
        elif row['nonNegDifCount2'] > 200:
            df.loc[index, 'nonNegDifCount2'] = 200
        if row['nonNegDigActivity'] < 0:
            df.loc[index, 'nonNegDigActivity'] = 0
        elif row['nonNegDigActivity'] > 200:
            df.loc[index, 'nonNegDigActivity'] = 200
        

    return df

def open_and_clean_IRC():
    df =sugar_bag_data["infrared_counters"]
    # Formatting time
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Check if data frame has null values
    if df.isnull().values.any():
        df.ffill( inplace = True)

    # Drop uneeded columns
    if 'battery' in df:
        df.drop(columns = ['battery'], inplace = True)
  
    # Remove duplicate entries
    df = df.drop_duplicates()

    # Remove invalid entries
    for index, row in df.iterrows():
        if row['peopleCount'] < 0:
            df.loc[index, 'peopleCount'] = 0
        if row["nonNegPeopleCount"] < 0:
            df.loc[index, 'nonNegPeopleCount'] = 0

    return df

def open_and_clean_VC(): 
    df =sugar_bag_data["'vibration_counters'"]

    # Formatting time
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Check if data frame has null values
    if df.isnull().values.any():
        df.ffill( inplace = True)

    # Drop uneeded columns
    if 'battery' in df:
        df.drop(columns = ['battery'], inplace = True)
    if 'batteryDiff' in df:
        df.drop(columns = ['batteryDiff'], inplace = True)
    if 'workCount' in df:
        df.drop(columns = ['workCount'], inplace = True)
        
    # Remove duplicate entries
    df = df.drop_duplicates()

    # Remove duplicate entries
    for index, row in df.iterrows():
        if row['workCountDiff'] < 0:
            df.loc[index, 'workCountDiff'] = 0

    return df

def open_and_clean_VC(): 
    df =sugar_bag_data["vibration_counters"]

    # Formatting time
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Check if data frame has null values
    if df.isnull().values.any():
        df.ffill( inplace = True)

    # Drop uneeded columns
    if 'battery' in df:
        df.drop(columns = ['battery'], inplace = True)
    if 'batteryDiff' in df:
        df.drop(columns = ['batteryDiff'], inplace = True)
    if 'workCount' in df:
        df.drop(columns = ['workCount'], inplace = True)
        
    # Remove duplicate entries
    df = df.drop_duplicates()

    # Remove duplicate entries
    for index, row in df.iterrows():
        if row['workCountDiff'] < 0:
            df.loc[index, 'workCountDiff'] = 0

    return df



TIME MANUPULATION 

In [79]:
"""Change the date or time
a string value that is formatted like the date, time, or both 
0000-00-00 00:00:00

an error with the code, is_valid_datetime is validating every date or time value, but if not full datetime given, then computer assigns a value when not wanted.
"""
def set_datetime(time):
    global local_time
    time = pd.to_datetime(time)
    date_part = local_time.date()
    time_part = local_time.time()

    
    if is_valid_datetime(time) == True:
        local_time = time
    elif is_valid_date(time) == True:
        date_part = time
        local_time =dt.combine(date_part, time_part)
    elif is_valid_time(time) == True:
        time_part = time
        local_time =dt.combine(date_part, time_part)
        print(local_time)
    else:
        print("""there is an error with your date imput. 
no changes made""")
        

"""Validates the given date, time, or both""" 
def is_valid_datetime(date_time, format="%Y-%m-%d %H:%M:%S"):
    date_timestr =date_time.strftime('%Y-%m-%d %H:%M:%S')
    try:
        # Try to parse the datetime string with the given format
        dt.strptime(date_timestr, format)
        print("ping")
        return True
    except ValueError:
        return False
        
def is_valid_date(date, format="%Y-%m-%d"):
    datestr =date.strftime('%H:%M:%S')
    try:
        # Try to parse the datetime string with the given format
        dt.strptime(datestr, format)
        return True
    except ValueError:
        return False
    
def is_valid_time(time, format="%H:%M:%S"):
    timestr =time.strftime('%H:%M:%S') #converts the timestamp into a string 
    try:
        # Try to parse the datetime string with the given format
        dt.strptime(timestr, format)
        return True
    except ValueError:
        return False


"""function to remove min and second from the time stamp
STATUS : INCOMPLETE
"""
def to_nearest_hr(time):


    
    pass

In [48]:
"""
find the distinct devices based on the dev_id

STATUS: WORKING
"""
def find_devices(df):
    dev_id_col = df["dev_id"]
    distinct_dev =[]

    for device in dev_id_col: 
        if device not in distinct_dev: 
            distinct_dev.append(device) 

    return distinct_dev


In [49]:
'''seperate the different devices into different columns.

STATUS: WORKING
'''
def seperate_devices(df):
    distinct =find_devices(df)
    split_list = {}

    for device in distinct:
        var_name = "%s" % device
        df_device = df.loc[df["dev_id"]==device].copy()
        df_device.drop(columns = ["dev_id"],inplace= True)
        split_list[var_name] = df_device
    
    #print("this is the fpc-6460 data ",split_list["fpc-6460"]) #check 
    return split_list


In [50]:
"""function to show from a selcected date for a given number of days, of a given datafrme
STATUS: WORKING
"""
def cut_view(df,start,span):
    start = pd.to_datetime(start)
    end = start + pd.DateOffset(days=span)
    #remove dates before
    filtered_df = df[df['datetime'] >= start]
    #remove dates after the given timeframe
    filtered_df = df[df['datetime'] <= end]

    return filtered_df


In [51]:
'''
ensures datetime is a value and readable

STATUS: WORKING UNUSED
'''
def format_DT(df):
     
    df['datetime'] = pd.to_datetime(df['datetime'])
    #df['Date']= pd.to_datetime(df['datetime']).dt.date
    #print(df["Date"].head())

    return df

def get_date(df):

    df['Date']= pd.to_datetime(df['datetime']).dt.date
    #print(df["Date"].head())


In [52]:
"""
Merges the columns based on the date, and sums each column for every day. 
works with dictionary

STATUS: WORKING
"""
def daily_totals(df):
    devices_inDict =seperate_devices(df)
    dict_key = list(devices_inDict.keys())
    daily = {}

    for i in dict_key:  
        total_PDay = devices_inDict[i].set_index('datetime').groupby(pd.Grouper(freq="D")).sum() #groups by the calander date'
        total_PDay['Date'] = total_PDay.index
        total_PDay["day_name"] = total_PDay["Date"].dt.day_name()
        daily[i]= total_PDay 

    
        print(daily[i])
    
    return daily


In [53]:
"""
merges cells based on the date by the frequency given in 'letter'
uses the grouper function's frequency 

STATUS: ASSUMED WORKING. not stress tested
"""
def totals(df,letter= "D"):
    devices_inDict =seperate_devices(df)
    dict_key = list(devices_inDict.keys())
    new = {}

    for i in dict_key:  
        total_PDay = devices_inDict[i].set_index('datetime').groupby(pd.Grouper(freq=letter)).sum() #groups by the calander date'
        new[i]= total_PDay 
        print(new[i])
    
    return new


In [54]:
"""
get a subplot based off the chosen day of the week

STATUS: INCOMPLETE
"""
def specific_day(df, day):
    df.drop(df[~(df['day_name'] == day)].index)
    print("new day column")
    print(df.head())
    return df


In [ ]:
"""create a dictionary with all the devices"""
def merger():
    
    weather = open_and_clean_Weather()
    weather_Clean = daily_totals(weather)
    
    WIFI = open_and_clean_WIFI()
    WIFI_Clean = daily_totals(WIFI)
    
    OBJ = open_and_clean_OBJ()
    OBJ_Clean = daily_totals(OBJ)
    
    IRBD = open_and_clean_IRBD()
    IRBD_Clean = daily_totals(IRBD)

    IRC = open_and_clean_IRC()
    IRC_Clean = daily_totals(IRC)
    
    """add names once formatted"""
    device_list = weather_Clean|WIFI_Clean|OBJ_Clean|IRBD_Clean|IRC_Clean
    
    return device_list

In [86]:
"""create a dictionary with all the devices"""
def device_dict():
    
    weather = open_and_clean_Weather()
    weather_Clean = seperate_devices(weather)
    
    WIFI = open_and_clean_WIFI()
    WIFI_Clean = seperate_devices(WIFI)
    
    OBJ = open_and_clean_OBJ()
    OBJ_Clean = seperate_devices(OBJ)
    
    IRBD = open_and_clean_IRBD()
    IRBD_Clean = seperate_devices(IRBD)

    IRC = open_and_clean_IRC()
    IRC_Clean = seperate_devices(IRC)
    
    """add names once formatted"""
    device_list = weather_Clean|WIFI_Clean|OBJ_Clean|IRBD_Clean|IRC_Clean
    
    return device_list


In [56]:
def clean_data():
    Weather = open_and_clean_Weather()

    WIFI = open_and_clean_WIFI()

    OBJ = open_and_clean_OBJ()

    IRBD = open_and_clean_IRBD()

    IRC = open_and_clean_IRC()

    VC = open_and_clean_VC()

    SB_Clean = {
    'weather_station': Weather,
    'wifi_counter':WIFI,
    'object_detection': OBJ,
    'infrared_bi_directional': IRBD,
    'infrared_counters': IRC,
    'vibration_counters': VC}


    return SB_Clean


Single values

In [75]:
def total_riders(): 
    global local_time
    data = clean_data()
    wificounter = data["wifi_counter"]

    if local_time < pd.to_datetime("2024-03-01 00:00:00"): #this negates a time error found before this date
        edited_time = local_time + pd.DateOffset(hours= 14)
    else:
        edited_time = local_time
    #get the current users. "current" or "total" to be used, total sometimes reather high and unsure if this is correct
    current_users = wificounter.loc[wificounter["datetime"] ==edited_time, "current"].values[0] 
    print("there are approx.", current_users, "people in the park")
    return current_users


In [93]:
"""test code to access how many riders have ridden the trail """
def trail_heads(trail,column): 
    global local_time
    time = local_time
    data = device_dict()
    iotdevice = data[trail]
    
    #infomation = data[trail].loc[:,"datetime" ]
    riders = iotdevice.loc[iotdevice["datetime"] ==time, column].values[0]
    print("there are approx.", riders, "on TRAIL NAME")
    return riders


local_time = pd.to_datetime("2023-01-21 10:00:00")
trail_heads("fpc-4f48",'nonNegDifCount1')


there are approx. 3.0 people in the park


3.0

In [102]:
def get_trail_numbers(trail_name): 
    #global local_time
    time = local_time
    trail_device = trail_info.loc[trail_info["Name"]== trail_name,"Device"].values[0]
    trail_count = trail_info.loc[trail_info["Name"]== trail_name,"Counter"].values[0]
    data = device_dict()
    iotdevice = data[trail_device]

    
   
    no_riders = iotdevice.loc[iotdevice["datetime"] ==time, trail_count].values[0]
    print("there are approx.", no_riders, "people on:", trail_name)
    #return riders
#get_trail_numbers("Bees Knees")

there are approx. 1.0 people on: Bees Knees


In [58]:
'''basic plotting function'''

def plotting(x, y, xlab ='x - axis', ylab ='y - axis', title='Graph Title'):
    
    plt.plot(x,y)       #plot the x and y values given
    plt.xlabel(xlab)    #lable the x vaule, a given value or default 'x - axis'
    plt.ylabel(ylab)    #lable the y vaule, a given value or default 'y - axis'
    plt.title(title)    #lable the graph, a given value or default 'Graph Title'
    plt.show()          #prints the graph in a window
    return 


In [59]:
def stuff():

    all_devices = merger()

    x = all_devices
    y = 0
    plotting(x,y)

TEST CASES

In [91]:
def test_datemod():
    global local_time
    print("the time is:",local_time)
    total_riders()
    set_datetime("2024-04-12")
    print("the time is:",local_time)
    total_riders()
    set_datetime("2024-04-12 12:00:00")
    print("the time is:",local_time)
    total_riders()
    set_datetime("17:00:00") #
    print("the time is:",local_time)
    total_riders()
    #set_datetime("2024-01-13 17:29:00") #will provide an error due to min value
    #total_riders()
local_time = pd.to_datetime("2023-01-15 10:00:00")
test_datemod()    

the time is: 2023-01-15 10:00:00


KeyboardInterrupt: 

In [84]:
#trail_heads("fpc-4f48","nonNegDifCount1")
aaaaaa = clean_data()
#bbbb = aaaaaa["fpc-4f48"]

In [31]:
data_SB = clean_data()
test_data = cut_view(data_SB['infrared_bi_directional'],"2023-01-15",3)
#pyg.walk(test_data)





there are approx. 17 people in the park
there are approx. 1 people in the park


1